In [3]:
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from importlib import reload
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
import scikitplot as skplt
import seaborn as sns
import matplotlib.pyplot as plt
from utility import load_data, plot_confusion_matrix, plot_average_f1_scores, train_model, get_classification_details, get_hard_disk_path, show_samples, plot_features_importance, visualize_correlation, get_shap
import utility
reload(utility)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "loss_function": "MultiClass",  # For multiclass classification
        "bootstrap_type": "Bernoulli",
        "silent": True
    }

    # Compute the sample weights
    sample_weights = compute_sample_weight(
        class_weight='balanced',
        y=y_train
    )

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train, sample_weight=sample_weights, eval_set=[(X_test, y_test)])

    predictions = model.predict(X_test)
    f1 = f1_score(y_test, predictions, average='macro')  # Use 'binary' for binary classification
    return f1


X, y, unique_labels, label_mapping, data = load_data(get_hard_disk_path("Segmentation"), "statistics_features_IP.csv")
# Use a list for labels to ensure consistent order
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the optimization results
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"{key}: {value}")

In [8]:
from importlib import reload
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
import scikitplot as skplt
import seaborn as sns
import matplotlib.pyplot as plt
from utility import load_data, plot_confusion_matrix, plot_average_f1_scores, train_model, get_classification_details, get_hard_disk_path, show_samples, plot_features_importance, visualize_correlation, get_shap
import utility
reload(utility)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

X, y, unique_labels, label_mapping, data = load_data(get_hard_disk_path("Segmentation"), "statistics_features_IP.csv")
# Use a list for labels to ensure consistent order
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print the optimization results
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"{key}: {value}")

[I 2024-02-19 11:13:38,120] A new study created in memory with name: no-name-f165f8d9-e001-401c-9171-b86781d10959


Successfully loaded data from D:/data_for_seg/
Successfully loaded data from E:/data_for_seg/


[I 2024-02-19 11:13:50,019] Trial 0 finished with value: 0.8779761904761905 and parameters: {'learning_rate': 0.03136771915050568, 'depth': 7, 'subsample': 0.41371079880528855, 'colsample_bylevel': 0.5790055229419003, 'min_data_in_leaf': 23}. Best is trial 0 with value: 0.8779761904761905.
[I 2024-02-19 11:14:19,960] Trial 1 finished with value: 0.7732142857142857 and parameters: {'learning_rate': 0.0036785253345102986, 'depth': 9, 'subsample': 0.27526495236061116, 'colsample_bylevel': 0.33998028633562155, 'min_data_in_leaf': 69}. Best is trial 0 with value: 0.8779761904761905.
[I 2024-02-19 11:15:23,100] Trial 2 finished with value: 0.7738095238095238 and parameters: {'learning_rate': 0.001511553345477651, 'depth': 9, 'subsample': 0.8951629990608516, 'colsample_bylevel': 0.8986166196312585, 'min_data_in_leaf': 60}. Best is trial 0 with value: 0.8779761904761905.
[I 2024-02-19 11:15:30,303] Trial 3 finished with value: 0.8446428571428571 and parameters: {'learning_rate': 0.016338926205

Number of finished trials:  100
Best trial:
Value:  0.8779761904761905
Params: 
learning_rate: 0.03136771915050568
depth: 7
subsample: 0.41371079880528855
colsample_bylevel: 0.5790055229419003
min_data_in_leaf: 23


In [12]:
# Import necessary libraries
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
import scikitplot as skplt
import shap

random_state = 42

X, y, unique_labels, label_mapping, data = load_data(get_hard_disk_path("Segmentation"), "statistics_features_IP.csv")
#visualize_correlation(X)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
ix_training, ix_test = [], []
# Loop through each fold and append the training & test indices to the empty lists above
for fold in kf.split(X, y):
    ix_training.append(fold[0]), ix_test.append(fold[1])
# List to store averaged SHAP values for each fold
average_shap_values_per_fold = []
cumulative_f1_scores = {label: 0 for label in unique_labels}

for i, (train_outer_ix, test_outer_ix) in enumerate(zip(ix_training, ix_test)): #-#-#
    X_train, X_test = X.iloc[train_outer_ix, :], X.iloc[test_outer_ix, :]
    y_train, y_test = y.iloc[train_outer_ix], y.iloc[test_outer_ix]
    classifier = train_model(X_train, y_train, unique_labels, "Catboost", best_params=study.best_trial)
    y_pred = classifier.predict(X_test)
    plot_confusion_matrix(y_test, y_pred, unique_labels)
    wellclassified_samples, misclassified_samples = get_classification_details(y_test, y_pred, data)
    #show_samples(get_hard_disk_path(), samples=wellclassified_samples, title="Correctly classified")
    #show_samples(get_hard_disk_path(), samples=misclassified_samples, title="Misclassified")
    shap_values = get_shap(classifier, X_test)
    # Now, you can plot the beeswarm chart
    # Check if SHAP values are a list (multi-class scenario)
    if isinstance(shap_values, list):
        # Summing SHAP values across all classes
        shap_values = np.sum([np.abs(sv) for sv in shap_values], axis=0)

    # Averaging SHAP values across samples for this fold
    average_shap_values_this_fold = np.mean(shap_values, axis=0)
    average_shap_values_per_fold.append(average_shap_values_this_fold)

    f1_scores_per_class = f1_score(y_test, y_pred, average=None)
    for idx, label in enumerate(unique_labels):
        cumulative_f1_scores[label] += f1_scores_per_class[idx]

# Averaging SHAP values across all folds
average_shap_values = np.mean(average_shap_values_per_fold, axis=0)

# Ensuring correct format for summary_plot
average_shap_values_2d = average_shap_values.reshape(1, -1)

# Plotting the heatmap
plt.figure(figsize=(10, 8))
shap.summary_plot(average_shap_values_2d, features=X.columns, plot_type="bar", show=False)
plt.tight_layout()
plt.show()
average_f1_scores = {label: score/5 for label, score in cumulative_f1_scores.items()} 
ranked_labels = sorted(average_f1_scores, key=average_f1_scores.get, reverse=True)
plot_average_f1_scores(ranked_labels, average_f1_scores)
plot_features_importance(classifier)

Successfully loaded data from D:/data_for_seg/
Successfully loaded data from E:/data_for_seg/


TypeError: catboost.core.CatBoostClassifier() argument after ** must be a mapping, not FrozenTrial